In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format
from scipy import io
from scipy.stats import entropy, kde, multivariate_normal, wasserstein_distance
from sklearn.metrics import mean_squared_error, r2_score, mutual_info_score, log_loss
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import sklearn.gaussian_process as gp
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
# from athena.active import ActiveSubspaces
import math
from sklearn.neighbors import KernelDensity
from scipy.spatial import KDTree
import random

In [ ]:
from core.GPR_build import GPR_fit
from core.preprocess import scale_data, inverse_transform
from core.estimate_pdf import estimate_pdf



In [ ]:
"""
Read the data
"""
## load CCF data of Jan and July at trg (all levels)
iRF_global = io.loadmat('data_for_training/iRF_global_trg.mat')
sorted(iRF_global.keys())

## load interpolated weather data of Jan and July at trg (all levels)
weather_global = io.loadmat('data_for_training/weather_global_trg.mat') #using dt=1, which is correct
sorted(weather_global.keys())


## Extract CCF values (observations)
iRF = iRF_global['iRF'].flatten('F')

# print(iRF)

## Extract meteorological features
temp = weather_global['tm1'].flatten('F')
geopot = weather_global['geopot'].flatten('F')
zi0 = weather_global['zi0'].flatten('F') #careful, redundancy
u = weather_global['u'].flatten('F')

## Extact latitudinal data (Release)
lon = np.linspace(-115,150,num=len(iRF),endpoint=True) 
lat = np.linspace(-50,85,num=len(iRF),endpoint=True)


## Create pandas dataframe

data = {"iRF":iRF, "temp":temp, "geopot":geopot,  \
        "zi0":zi0,  \
        "u":u,  "lat":lat}

data = pd.DataFrame(data)


data_LR = {"iRF":iRF, "temp":temp, "geopot":geopot,  \
          "Tphi":temp*geopot}

data_LR = pd.DataFrame(data_LR)



X = data.drop(["iRF"], 1) #choose all features for GPR
X2 = data_LR.drop(["iRF"], 1) #choose original features for LR
y = data[["iRF"]]

"""
Check contents and shape of data
"""
print(X.shape,y.shape)
print(data)

In [ ]:
"""
Detect outlier(s) using Tukey's method
"""

# The data to be analysed
yd = y


q1 = np.quantile(yd, 0.25) #1st quartile
q3 = np.quantile(yd, 0.75) #3rd quartile
med = np.median(yd)
 

iqr = q3-q1 #Inter quartile Range (IQR)
 
lb = q1 - (1.5*iqr) #lower bound
ub = q3 + (1.5*iqr) #upper bound

# print(lb, ub, med, np.mean(np.asarray(yd)))

# finding outliers
outliers = np.asarray(yd)[(np.asarray(yd) <= lb) | (np.asarray(yd) >= ub)]
outliers_round = np.round(outliers,8)
print(f'The following are the outliers in the boxplot:{outliers_round}')

idx_outliers = np.where(np.isin(y, outliers.flatten()))
idx_outliers = np.squeeze(idx_outliers)[0]
print(f'The corresponding indices are:{idx_outliers}')

#Boxplot
_ = plt.figure(figsize=(6,4))
plt.boxplot(yd)
plt.xlabel('')
plt.ylabel(r'iRF [W/m$^2$]',fontsize=14)
plt.tick_params(axis='y', labelsize=14)
plt.show()

In [ ]:
"""
Outliers must only be removed if there is an expected error
"""

outlier_removal = False 

if outlier_removal:
    y = np.delete(y.values, idx_outliers)
    X = np.delete(X.values, idx_outliers, axis=0)
    X2 = np.delete(X2.values, idx_outliers, axis=0)
    print(y.shape,X.shape)

In [ ]:
"""
Split the data into train (80%) and test (20%) groups
"""

#Randomly split the data  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y, test_size = 0.2, random_state = 0)

In [ ]:
"""
Scale the data matrix (X,y) are of the same order of magnitude
"""


if outlier_removal:
    for var in [y_train, y_train2, y_test, y_test2]:
        var.shape = (-1, 1)  # Reshape in-place


# Scale both datasets
X_train, X_test, y_train, y_test, scalerX, scalery = scale_data(X_train, X_test, y_train, y_test)
X_train2, X_test2, y_train2, y_test2, scalerX2, scalery2 = scale_data(X_train2, X_test2, y_train2, y_test2)

print(y_train[:5])

In [ ]:
"""
Fit the standard GPR model to the data
"""

y_pred_test, y_std, y_samples, model = GPR_fit(X_train, y_train, X_test)

print(model.kernel_)

In [ ]:
print(y_pred_test.shape, y_samples.shape, X_test.shape)

In [ ]:
"""
Visualise the Kernel Density Estimate (KDE) of the test data and predictions from the standard GPR model
"""
## Build KDE for all samples generated by GPR model since it contains variance information, 
##  not just the mean
p_y_data, p_y_gp, test = estimate_pdf(y1=y_test,y2=y_samples,pts=2000,plot_fig=True,label1='observations',label2='predictions')

In [ ]:
"""
Put back the data on the original scale
"""

# Inverse transform first set
y_train_inverse, y_pred_inverse, y_test_inverse = inverse_transform(scalery, y_train, y_pred_test, y_test)
X_train_inverse, X_test_inverse = inverse_transform(scalerX, X_train, X_test)
y_std_inverse, y_samples_inverse = inverse_transform(scalery, y_std, y_samples)

print(y_samples_inverse.shape)

# Inverse transform second set
y_train_inverse2, y_test_inverse2 = inverse_transform(scalery2, y_train2, y_test2)
X_train_inverse2, X_test_inverse2 = inverse_transform(scalerX2, X_train2, X_test2)

In [ ]:
"""
Comparing the data and predictions from the standard GPR model
- R2
- KL-divergence
"""


## Scatter plot
fig = plt.figure(figsize=(8,8))
plt.rcParams['font.family'] = 'Helvetica'
plt.scatter(y_test_inverse, y_pred_inverse, color='red', s = 20)
plt.plot(y_test_inverse, y_test_inverse, 'k')
plt.xlabel('CCM iRF [W/m$^2$]')
plt.ylabel('Predicted iRF [W/m$^2$]')
plt.title('Standard GP')
plt.axis('square')
# plt.savefig('GPR_fit.png', dpi=300, bbox_inches='tight')
print("R\u00b2 score of GPR is = ",r2_score(y_test_inverse, y_pred_inverse))
print("KL-div(pred,obs) is = ",entropy(p_y_gp, p_y_data)) #asymmetric 'metric'


In [ ]:
"Apply Linear Regression - Manen and Grewe's features"

sk_model = LinearRegression()
sk_model.fit(X_train2, y_train2)
beta0 = sk_model.intercept_
beta = sk_model.coef_
# y_pred_test_LR = sk_model.predict(X_train2)
y_pred_test_LR = sk_model.predict(X_test2)
y_pred_LR_inverse = scalery2.inverse_transform(y_pred_test_LR.reshape(-1,1))
print(beta)

In [ ]:
np.array_equal(y_test_inverse, y_test_inverse2) #this should be equal

In [ ]:
"Apply Linear Regression - Manen and Grewe's features"

maroon = (212/255, 17/255, 89/255)
blue = (26/255, 133/255, 255/255) 

plt.figure(figsize=(8,8))

plt.scatter(y_test_inverse, y_pred_inverse, color = maroon, s = 20, label = 'Standard GP, R$^2$ = 0.53')
plt.scatter(y_test_inverse2, y_pred_LR_inverse, color = blue, s = 20, label = 'Linear Regression, R$^2$ = 0.05')
plt.plot(y_test_inverse, y_test_inverse, 'k')
plt.xlabel('CCM iRF [W m$^{-2}$]')
plt.ylabel('Predicted iRF [W m$^{-2}$]')
# plt.title("R\u00b2 score of GPR is = ", round(r2_score(y_test_inverse, y_pred_inverse),2))
plt.legend(fontsize=15)
plt.axis('square')
plt.tight_layout()
# plt.savefig('GPR_LR_MG_features.png', dpi=300, bbox_inches='tight')

print("R\u00b2 score of GPR is = ", round(r2_score(y_test_inverse, y_pred_inverse),2))
print("R\u00b2 score of LR is = ", round(r2_score(y_test_inverse, y_pred_LR_inverse),2))

# Train and test the heteroscedastic GPR

#### Some common packages are imported again if you want to make a separate notebook

In [ ]:
import seaborn as sns
sns.set_theme(style="white")
sns.set_context("talk")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import os
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import gpflow as gpf

from gpflow.ci_utils import reduce_in_tests
from gpflow.utilities import print_summary
gpf.config.set_default_summary_fmt("notebook")
optimizer_config = dict(maxiter=reduce_in_tests(1000))

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from core.GPR_build import train_chainedGP

In [ ]:
# likelihood, kernel, inducing_variable = build_chainedGP(X_train, n_latents=2, M_offset=10, kernel_variance=1.0)


In [ ]:
"Declare likelihood, latent functions for mean and variance for chained GPR model"

############################################################## likelihood

likelihood = gpf.likelihoods.HeteroskedasticTFPConditional(
    distribution_class=tfp.distributions.Normal,  # Gaussian Likelihood
#     distribution_class=tfp.distributions.LogNormal(),  # 
    scale_transform=tfp.bijectors.Softplus(),  # (Softplus Transform ()originally: Exp Transform. option: Softplus)
)
############################################################## kernel
kernel = gpf.kernels.SeparateIndependent(
    [
        gpf.kernels.SquaredExponential(lengthscales=np.ones(X_train.shape[1]),variance=1.0),  # This is k1, the kernel of f1
        gpf.kernels.SquaredExponential(variance=1.0),    # this is k2, the kernel of f2
    ]
)
############################################################## inducing_variable
M = len(X_train)-10 # this can be any value less than len(X_train). Only these many points are used to parametrize the covariance matrix.
Z = KMeans(n_clusters=M).fit(X_train).cluster_centers_

inducing_variable = gpf.inducing_variables.SeparateIndependentInducingVariables(
    [
        gpf.inducing_variables.InducingPoints(Z),  # This is Uf = f(Z)
        gpf.inducing_variables.InducingPoints(Z),  # This is Ug = g(Z)
    ]
)

In [ ]:
print(f"Likelihood's expected latent_dim: {likelihood.latent_dim}") #Number of latent functions 

In [ ]:
X_train, X_test, y_train, y_test = map(pd.DataFrame, [X_train, X_test, y_train, y_test])


In [ ]:
"Train the chained GPR model with multiple epochs"

epochs = 800
alpha = 0.01
#n = 0
plt.figure()
model, loss = train_chainedGP(X_train, y_train, kernel, likelihood, inducing_variable, epochs, alpha)
sns.lineplot(np.arange(0, epochs), loss)
plt.xlabel("Epochs")
plt.ylabel("Loss")
sns.despine()
print_summary(model)

In [ ]:
"Mean and variance of model predictions over the test set"

mean_y, var_y = model.predict_y(X_test.values)
sigma_y = np.sqrt(var_y)

n_samples = 5000
chained_samples_y = []
for i in range(len(X_test.values)):
    chained_samples_y.append(np.random.normal(mean_y[i], sigma_y[i], n_samples))
    
chained_samples_flat = [item for sublist in chained_samples_y for item in sublist] # flatten the list


In [ ]:
print(mean_y.shape,var_y.shape)

In [ ]:
print(np.shape(chained_samples_y))

In [ ]:
print(len(chained_samples_y), len(chained_samples_flat))

chained_samples_y = np.asarray(chained_samples_y).T

In [ ]:
print(np.shape(chained_samples_y), np.shape(chained_samples_flat))

In [ ]:
y_pred_chained = []

for i in range(0, len(X_test)):
    y_pred_chained = np.hstack((y_pred_chained, chained_samples_y[:,i]))


In [ ]:
print(np.shape(y_pred_chained))

In [ ]:
"Placing the predictions on the real scale"

y_pred_chained_inverse = scalery.inverse_transform(y_pred_chained.reshape(-1,1))
y_pred_chained_mean = np.mean(y_pred_chained.reshape(-1, n_samples), axis=1)
y_pred_chained_mean_inverse = np.mean(y_pred_chained_inverse.reshape(-1, n_samples), axis=1)

In [ ]:
"Build KDE for all samples generated by both GP models since they contain variance information,not just the mean"

p_y_standard, p_y_chained, test = estimate_pdf(y1=y_samples, y2=y_pred_chained, pts=2000, plot_fig=False, label1='standard GPR', label2='chained GPR')

In [ ]:
print(np.mean(y_pred_chained_inverse.reshape(-1,1)),np.std(y_pred_chained_inverse.reshape(-1,1)))

In [ ]:
"Evaluate performance based on certain aspects"

print("WD1(pred,obs) for standard model is = ", np.round(wasserstein_distance(y_test.values.reshape(-1), y_samples.reshape(-1)), 3)) #symmetric metric
print("KL-div(pred,obs) for standard model is = ",np.round(entropy(p_y_standard, p_y_data), 3)) #not symmetric!
print("R\u00b2 score of standard model is = ",np.round(r2_score(y_test, y_pred_test),2))

fig = plt.figure()
plt.tick_params(left = True, right = False , labelleft = True ,
                labelbottom = False, bottom = False)
plt.plot(test,p_y_data,color="blue",label="CCM data")
plt.plot(test,p_y_standard,color="red",label="Standard GP")
plt.plot(test,p_y_chained,color="green",label="Chained GP")

plt.xlabel('$y{_*}$')
plt.ylabel('Density')
plt.legend(fontsize=15)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
# plt.savefig(f'model_compare.png', dpi=300, bbox_inches='tight') 


print("WD1(pred,obs) for chained model is = ", np.round(wasserstein_distance(y_test.values.reshape(-1), y_pred_chained), 3)) #symmetric 
print("KL-div(pred,obs) for chained model is = ",np.round(entropy(p_y_chained, p_y_data), 3)) #not!
print("R\u00b2 score of chained model is = ", np.round(r2_score(y_test, y_pred_chained_mean),2))


#### Figure 3a: Chained GPR mean predictions against test data


In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_test_inverse, y_pred_chained_mean_inverse, color = "teal", s = 20, label = 'Chained GP')
plt.plot(y_test_inverse, y_test_inverse, 'k')
plt.xlabel('CCM iRF [W m$^{-2}$]')
plt.ylabel('Predicted iRF [W m$^{-2}$]')
plt.title('Chained GP')
plt.axis('square')
plt.savefig('GPR_fit_chained.png', dpi=300, bbox_inches='tight')

#### Violin plots in Figure 3b and supplementary material

In [ ]:
print(f'The following are the outliers in the boxplot:{outliers_round}')

test_outliers = np.where(np.isin(y_test_inverse, outliers.flatten()))
test_outliers = np.squeeze(test_outliers)[0]
print(f'The corresponding indices are:{test_outliers}')

random.seed(4) #used for Figure 3(b)
z = random.sample(list(range(0, len(X_test))), 20)
print(f'The randomly chosen test indices are = {z}') #one of the outliers get randomly included in this set

z[-5:] = test_outliers[-5:] #replcaing the last five entries with remianing outliers for later visualisation


z[3], z[-6] = z[-6], z[3] #swapping positions to include all outliers at the end

print(f'The randomly chosen test indices including all test outliers are = {z}')
print(f'Sanity check: {z[-6:]}')

In [ ]:
# Feature indices and labels
fidx = range(X_test.shape[1])
flist = [
    "Temperature [K]",
    "Geopotential [m$^2$ s$^{-2}$]",
    r"Solar irradiance [W m$^{-2}$]",
    "Wind velocity [m s$^{-1}$]",
    r"Release location [$^{\circ}$]"
]

# Reshape prediction data
data = y_pred_chained_inverse.reshape(len(X_test), n_samples)[z].T  # Chained GPR
data2 = y_samples_inverse.reshape(len(X_test), n_samples)[z].T      # Standard GPR

# Variance for color mapping
variances = np.std(data, axis=0)
variances2 = np.std(data2, axis=0)

# Color maps
cmap = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(variances), vmax=max(variances)))

# Violin widths based on feature scale
widths = [np.mean(np.abs(np.diff(X_test_inverse[:, i]))) / 2 for i in range(X_test.shape[1])]

# Plot per feature
for j in fidx:
    fig, axs = plt.subplots(figsize=(16, 7))

    # ======= STANDARD GPR =======
    ax1 = plt.subplot(1, 2, 1)
    ax1.set_title('Standard GP')

    ax1.scatter(X_test_inverse[z, j], y_test_inverse[z], s=12, color="orange", label="CCM data")
    ax1.scatter(X_test_inverse[z[-6:], j], y_test_inverse[z[-6:]], marker='s', s=100, facecolors='none', edgecolors='magenta')
    ax1.scatter(X_test_inverse[z, j], y_pred_inverse[z], s=12, color="blue", label="mean prediction")

    violins_std = ax1.violinplot(data2, X_test_inverse[z, j], widths=widths[j], showextrema=False, points=n_samples, bw_method='scott')

    for i, pc in enumerate(violins_std['bodies']):
        pc.set_edgecolor('black')
        pc.set_facecolor(cmap.to_rgba(variances2[i]))

    cbar_std = plt.colorbar(cmap, ax=ax1)
    cbar_std.set_label('Variance [W$^2$ m$^{-4}$]')

    ax1.set_xlabel(flist[j])
    ax1.set_ylabel("iRF [W m$^{-2}$]")
    ax1.set_ylim(top=1e-4)
    ax1.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
    ax1.legend(loc="upper center", fontsize=14)

    # ======= CHAINED GPR =======
    ax2 = plt.subplot(1, 2, 2)
    ax2.set_title('Chained GP')

    ax2.scatter(X_test_inverse[z, j], y_test_inverse[z], s=12, color="orange", label="CCM data")
    ax2.scatter(X_test_inverse[z[-6:], j], y_test_inverse[z[-6:]], marker='s', s=100, facecolors='none', edgecolors='magenta')
    ax2.scatter(X_test_inverse[z, j], np.mean(y_pred_chained_inverse.reshape(-1, n_samples), axis=1)[z], s=12, color="black", label="mean prediction")

    violins_chain = ax2.violinplot(data, X_test_inverse[z, j], widths=widths[j], showextrema=False, points=n_samples, bw_method='scott')

    for i, pc in enumerate(violins_chain['bodies']):
        pc.set_edgecolor('black')
        pc.set_facecolor(cmap.to_rgba(variances[i]))

    cbar_chain = plt.colorbar(cmap, ax=ax2)
    cbar_chain.set_label('Variance [W$^2$ m$^{-4}$]')

    ax2.set_xlabel(flist[j])
    ax2.set_ylabel("iRF [W m$^{-2}$]")
    ax2.set_ylim(top=1e-4)
    ax2.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
    ax2.legend(loc="best", fontsize=14)

    plt.tight_layout(pad=2.0)
    # plt.savefig(f'violinplot_{j}.png', dpi=400, bbox_inches='tight')
    plt.show()



### Climate impact prediction of arbitrary flight paths

In [ ]:
from core.load_trajectory_data import load_trajectory_data 

In [ ]:
"Load trajectory data from EUROCONTROL and AirTraf"

trajectory_info = [
    ('traj1.mat', '1', 'flight_latitude_f1'),
    ('traj2.mat', '2', 'flight_latitude_f2'),
    ('traj3.mat', '3', 'flight_latitude_f3'),
    ('traj4_AirTraf.mat', '4', 'flight_latitude_f4')
]


X_traj1, X_traj2, X_traj3, X_traj4 = [
    load_trajectory_data(os.path.join("trajectory_data", fname), suffix, lat_key)
    for fname, suffix, lat_key in trajectory_info
]

print(X_traj1.shape, X_traj2.shape, X_traj3.shape, X_traj4.shape) #AirTraf trajectory will have 101 points

In [ ]:
X_traj4.t_4

In [ ]:
"Remove take off and landing part for AirTraf"

t_4 = X_traj4.t_4[12:-11]
g_4 = X_traj4.g_4[12:-11]
zi0_4 = X_traj4.zi0_4[12:-11]
u_4 = X_traj4.u_4[12:-11]
Rl_4 = X_traj4.Rl_4[12:-11]

print(Rl_4)

#interpolate to size of other flights

t_4 = np.interp(np.linspace(0, 1, X_traj1.shape[0]), np.linspace(0, 1, len(t_4)), t_4)
g_4 = np.interp(np.linspace(0, 1, X_traj1.shape[0]), np.linspace(0, 1, len(g_4)), g_4)
zi0_4 = np.interp(np.linspace(0, 1, X_traj1.shape[0]), np.linspace(0, 1, len(zi0_4)), zi0_4)
u_4 = np.interp(np.linspace(0, 1, X_traj1.shape[0]), np.linspace(0, 1, len(u_4)), u_4)
Rl_4 = np.interp(np.linspace(0, 1, X_traj1.shape[0]), np.linspace(0, 1, len(Rl_4)), Rl_4)

print(Rl_4)

In [ ]:
X_traj4 = {"t_4":t_4, "g_4":g_4,  \
        "zi0_4":zi0_4,  \
        "u_4":u_4,  "Rl_4":Rl_4}

X_traj4 = pd.DataFrame(X_traj4)

print(X_traj1.shape, X_traj2.shape, X_traj3.shape, X_traj4.shape) #they are all of equal length

In [ ]:
"Applying the same logic as before and predicting the climate impact of the trajectory data"

scalerXtraj1 = StandardScaler().fit(X_traj1)
X_traj1 = scalerXtraj1.transform(X_traj1)
scalerXtraj2 = StandardScaler().fit(X_traj2)
X_traj2 = scalerXtraj2.transform(X_traj2)
scalerXtraj3 = StandardScaler().fit(X_traj3)
X_traj3 = scalerXtraj3.transform(X_traj3)
scalerXtraj4 = StandardScaler().fit(X_traj4)
X_traj4 = scalerXtraj4.transform(X_traj4)

mean_ytraj1, var_ytraj1 = model.predict_y(X_traj1)
sigma_ytraj1 = np.sqrt(var_ytraj1)

mean_ytraj2, var_ytraj2 = model.predict_y(X_traj2)
sigma_ytraj2 = np.sqrt(var_ytraj2)

mean_ytraj3, var_ytraj3 = model.predict_y(X_traj3)
sigma_ytraj3 = np.sqrt(var_ytraj3)

mean_ytraj4, var_ytraj4 = model.predict_y(X_traj4)
sigma_ytraj4 = np.sqrt(var_ytraj4)


chained_samples_ytraj1 = []
chained_samples_ytraj2 = []
chained_samples_ytraj3 = []
chained_samples_ytraj4 = []

#length of feature vector is not the same for all trajectories!
for i in range(len(X_traj1)):
    chained_samples_ytraj1.append(np.random.normal(mean_ytraj1[i], sigma_ytraj1[i], n_samples))
    
for i in range(len(X_traj2)):
    chained_samples_ytraj2.append(np.random.normal(mean_ytraj2[i], sigma_ytraj2[i], n_samples))
    
for i in range(len(X_traj3)):
    chained_samples_ytraj3.append(np.random.normal(mean_ytraj3[i], sigma_ytraj3[i], n_samples)) 
    
for i in range(len(X_traj4)):
    chained_samples_ytraj4.append(np.random.normal(mean_ytraj4[i], sigma_ytraj4[i], n_samples))     
    
    
chained_samples_ytraj1 = np.asarray(chained_samples_ytraj1).T
chained_samples_ytraj2 = np.asarray(chained_samples_ytraj2).T
chained_samples_ytraj3 = np.asarray(chained_samples_ytraj3).T
chained_samples_ytraj4 = np.asarray(chained_samples_ytraj4).T

y_pred_traj1 = []
y_pred_traj2 = []
y_pred_traj3 = []
y_pred_traj4 = []

#length of feature vector is not the same for all trajectories!
for i in range(0, len(X_traj1)):
    y_pred_traj1 = np.hstack((y_pred_traj1, chained_samples_ytraj1[:,i]))
    
for i in range(0, len(X_traj2)):
    y_pred_traj2 = np.hstack((y_pred_traj2, chained_samples_ytraj2[:,i]))

for i in range(0, len(X_traj3)):
    y_pred_traj3 = np.hstack((y_pred_traj3, chained_samples_ytraj3[:,i]))   

for i in range(0, len(X_traj4)):
    y_pred_traj4 = np.hstack((y_pred_traj4, chained_samples_ytraj4[:,i]))       

In [ ]:
X_traj1_inverse = scalerXtraj1.inverse_transform(X_traj1)
X_traj2_inverse = scalerXtraj2.inverse_transform(X_traj2)
X_traj3_inverse = scalerXtraj3.inverse_transform(X_traj3)
X_traj4_inverse = scalerXtraj4.inverse_transform(X_traj4)

In [ ]:
#The impact is based on the fact that 5e5 kg(NO) is released per trg in the CCM simulation. This should be converted to kg(NO2)
#by using the molecular weight ratio of NO2 to NO, which makes it 7.665e5 kg(NO2). Thus our prediction is W/m2/kg(NO2).

scale_factor = 7.665e5
y_pred_traj1_inverse = scalery.inverse_transform(y_pred_traj1.reshape(-1,1))/scale_factor
y_pred_traj2_inverse = scalery.inverse_transform(y_pred_traj2.reshape(-1,1))/scale_factor
y_pred_traj3_inverse = scalery.inverse_transform(y_pred_traj3.reshape(-1,1))/scale_factor
y_pred_traj4_inverse = scalery.inverse_transform(y_pred_traj4.reshape(-1,1))/scale_factor
y_mean_traj1_inverse = scalery.inverse_transform(mean_ytraj1.reshape(-1,1))/scale_factor
y_mean_traj2_inverse = scalery.inverse_transform(mean_ytraj2.reshape(-1,1))/scale_factor
y_mean_traj3_inverse = scalery.inverse_transform(mean_ytraj3.reshape(-1,1))/scale_factor
y_mean_traj4_inverse = scalery.inverse_transform(mean_ytraj4.reshape(-1,1))/scale_factor


y_sigma_traj1_inverse = sigma_ytraj1.reshape(-1,1)*scalery.scale_/scale_factor
y_sigma_traj2_inverse = sigma_ytraj2.reshape(-1,1)*scalery.scale_/scale_factor
y_sigma_traj3_inverse = sigma_ytraj3.reshape(-1,1)*scalery.scale_/scale_factor
y_sigma_traj4_inverse = sigma_ytraj4.reshape(-1,1)*scalery.scale_/scale_factor

#### Figure 5 (a)

In [ ]:
"Visualising the impact of the trajectories"

fig, axes = plt.subplots(figsize=(10,6))
plt.plot(X_traj4_inverse[:,-1],np.mean(y_pred_traj2_inverse.reshape(-1,n_samples),axis=1), color="#000000", marker='s', label="Flight 1")
plt.plot(X_traj4_inverse[:,-1],np.mean(y_pred_traj1_inverse.reshape(-1,n_samples),axis=1), color="#1E88E5", marker='o', label="Flight 2")
plt.plot(X_traj4_inverse[:,-1],np.mean(y_pred_traj3_inverse.reshape(-1,n_samples),axis=1), color="#D81B60", marker='^',label="Flight 3")
plt.plot(X_traj4_inverse[:,-1],np.mean(y_pred_traj4_inverse.reshape(-1,n_samples),axis=1), color="#FFC107", linestyle='dashed', label="Great circle")
plt.ylabel("iRF [W m$^{-2}$ kg(NO$_2$)$^{-1}$]")
plt.xlabel("Latitude")
# plt.xlabel("Cruise flight completion (%)")
plt.fill_between(
    X_traj4_inverse[:,-1],
#     np.linspace(0,100,18).round()
    (y_mean_traj1_inverse.ravel() - y_sigma_traj1_inverse.ravel()),
    (y_mean_traj1_inverse.ravel() + y_sigma_traj1_inverse.ravel()),
    color="#1E88E5", alpha=0.2,
)
plt.fill_between(
    X_traj4_inverse[:,-1],
#     np.linspace(0,100,18).round()
    (y_mean_traj2_inverse.ravel() - y_sigma_traj2_inverse.ravel()),
    (y_mean_traj2_inverse.ravel() + y_sigma_traj2_inverse.ravel()),
    color="#000000", alpha=0.2,
)
plt.fill_between(
    X_traj4_inverse[:,-1],
#     np.linspace(0,100,18).round()
    (y_mean_traj3_inverse.ravel() - y_sigma_traj3_inverse.ravel()),
    (y_mean_traj3_inverse.ravel() + y_sigma_traj3_inverse.ravel()),
    color="#D81B60", alpha=0.2,
)
plt.fill_between(
    X_traj4_inverse[:,-1],
#     np.linspace(0,100,18).round()
    (y_mean_traj4_inverse.ravel() - y_sigma_traj4_inverse.ravel()),
    (y_mean_traj4_inverse.ravel() + y_sigma_traj4_inverse.ravel()),
    color="#FFC107", alpha=0.2,
)
# axes.invert_xaxis()
# plt.xticks(np.arange(0, 18, 1))
xtick_positions = [51, 45, 42]  # Specify the positions where you want ticks
xtick_labels = [r'50$^{\circ}$N', '45$^{\circ}$N', '42$^{\circ}$N']  # Specify corresponding labels
# xtick_positions = [0, 12, 17]  # Specify the positions where you want ticks
# xtick_labels = ['0%', '71%', '100%']  # Specify corresponding labels
# Set the custom x-axis ticks
plt.xticks(xtick_positions, xtick_labels)

plt.legend(fontsize=14, loc='upper center')

# plt.savefig(f'iRF_prediction_eurocontrol.png', dpi=300, bbox_inches='tight') 
